In [701]:
#This solution is incorrect. There is a need for further evalution of how to build the exponential trend with seasonality.

In [1]:
%matplotlib inline

from pathlib import Path

import math
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import statsmodels.formula.api as sm
from statsmodels.tsa import tsatools, stattools
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics import tsaplots

In [2]:
deptstore = pd.read_csv('DepartmentStoreSales.csv')

In [5]:
deptstore['Quarter'] = 'Q' + deptstore.Quarter.astype('str')
date = deptstore.Quarter.str.replace(r'(Q\d)-(\d+)')
deptstore['Date'] = pd.to_datetime(date).dt.strftime('%Q')

TypeError: replace() missing 1 required positional argument: 'repl'

In [705]:
deptstore

,Quarter,Sales
0,Q1,50147
1,Q2,49325
2,Q3,57048
3,Q4,76781
4,Q5,48617
5,Q6,50898
6,Q7,58517
7,Q8,77691
8,Q9,50862
9,Q10,53028


In [706]:
len(deptstore)

24

In [707]:
deptstore_ts = pd.Series(deptstore.Sales.values, name = 'Sales')
deptstore_df = tsatools.add_trend(deptstore_ts, trend = 'ct')
deptstore_df['Quarter'] = deptstore.Quarter

In [708]:
deptstore_df

,Sales,const,trend,Quarter
0,50147,1.0,1.0,Q1
1,49325,1.0,2.0,Q2
2,57048,1.0,3.0,Q3
3,76781,1.0,4.0,Q4
4,48617,1.0,5.0,Q5
5,50898,1.0,6.0,Q6
6,58517,1.0,7.0,Q7
7,77691,1.0,8.0,Q8
8,50862,1.0,9.0,Q9
9,53028,1.0,10.0,Q10


In [709]:
nValid = 4
nTrain = len(deptstore_ts) - nValid

#partition the data
train_df = deptstore_df[:nTrain]
valid_df = deptstore_df[nTrain:]

In [710]:
train_df

,Sales,const,trend,Quarter
0,50147,1.0,1.0,Q1
1,49325,1.0,2.0,Q2
2,57048,1.0,3.0,Q3
3,76781,1.0,4.0,Q4
4,48617,1.0,5.0,Q5
5,50898,1.0,6.0,Q6
6,58517,1.0,7.0,Q7
7,77691,1.0,8.0,Q8
8,50862,1.0,9.0,Q9
9,53028,1.0,10.0,Q10


In [711]:
valid_df

,Sales,const,trend,Quarter
20,60800,1.0,21.0,Q21
21,64900,1.0,22.0,Q22
22,76997,1.0,23.0,Q23
23,103337,1.0,24.0,Q24


In [712]:
deptstore_lm_expo_season = sm.ols(formula = 'np.log(Sales) ~ trend + Quarter', data = train_df).fit()
deptstore_lm_expo_season.summary()

C:\Users\GETHSEMAN\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1698: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\GETHSEMAN\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1699: RuntimeWarning: invalid value encountered in double_scalars
  * (1 - self.rsquared))
C:\Users\GETHSEMAN\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\GETHSEMAN\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\GETHSEMAN\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\Users\GETHSEMAN\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1620: Runt

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(Sales)   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 13 Jun 2020   Prob (F-statistic):                nan
Time:                        19:55:32   Log-Likelihood:                 579.71
No. Observations:                  20   AIC:                            -1119.
Df Residuals:                       0   BIC:                            -1099.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         10.7986        inf          0        nan         nan         nan
Quarter[T.Q10]    -0.1609        inf         -0        nan         nan         nan
Quarter[T.Q11]    -0.0809        inf         -0        nan         nan         nan
Quarter[T.Q12]     0.1978        inf          0        nan         nan         nan
Quarter[T.Q13]    -0.2597        inf         -0        nan         nan         nan
Quarter[T.Q14]    -0.2369        inf         -0        nan         nan         nan
Quarter[T.Q15]    -0.0674        inf         -0        nan         nan         nan
Quarter[T.Q16]     0.1684        inf          0        nan         nan         nan
Quarter[T.Q17]    -0.2678        inf         -0        nan         nan         nan
Quarter[T.Q18]    -0.2296        inf         -0        nan         nan         nan
Quarter[T.Q19]    -0.0790        inf         -0        nan         nan         nan
Quarter[T.Q2]     -0.0406        inf         -0        nan         nan         nan
Quarter[T.Q20]     0.1511        inf          0        nan         nan         nan
Quarter[T.Q3]      0.0808        inf          0        nan         nan         nan
Quarter[T.Q4]      0.3537        inf          0        nan         nan         nan
Quarter[T.Q5]     -0.1273        inf         -0        nan         nan         nan
Quarter[T.Q6]     -0.1056        inf         -0        nan         nan         nan
Quarter[T.Q7]      0.0098        inf          0        nan         nan         nan
Quarter[T.Q8]      0.2692        inf          0        nan         nan         nan
Quarter[T.Q9]     -0.1785        inf         -0        nan         nan         nan
trend              0.0241        inf          0        nan         nan         nan
==============================================================================
Omnibus:                        0.960   Durbin-Watson:                   0.289
Prob(Omnibus):                  0.619   Jarque-Bera (JB):                0.751
Skew:                          -0.075   Prob(JB):                        0.687
Kurtosis:                       2.063   Cond. No.                         136.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The input rank is higher than the number of observations.
"""